In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from keras.layers import Dense,Dropout
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso,Ridge
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.layers.advanced_activations import ReLU, PReLU
from keras.optimizers import SGD, Adam
from scipy.stats import mode
from keras.applications.vgg16 import VGG16
from keras.layers import Dense,Dropout
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.optimizers import SGD, Adam
from keras.layers.advanced_activations import ReLU, PReLU

In [165]:
train_x = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\train_x_xgboost.csv')
train_y = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\train_y_xgboost.csv')
test_x = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\test_x_xgboost.csv')

In [76]:
train_y_poutcome_dummies

,0,1,2,3
0,0,1,0,0
1,0,0,0,1
2,1,0,0,0
3,1,0,0,0
4,0,0,0,1
...,...,...,...,...
27095,0,0,0,1
27096,0,0,0,1
27097,0,0,0,1
27098,0,0,0,1


In [78]:
train_x_poutcome_xg = train_x.drop(['poutcome'], axis=1)
train_y_poutcome_xg = train_x['poutcome']
train_y_poutcome_dummies = pd.get_dummies(train_x['poutcome'])
train_y_poutcome_f = train_y_poutcome_dummies[0]
train_y_poutcome_o = train_y_poutcome_dummies[1]
train_y_poutcome_s = train_y_poutcome_dummies[2]
train_y_poutcome_u = train_y_poutcome_dummies[3]
test_x_poutcome_xg = test_x.drop(['poutcome'], axis=1)
test_y_poutcome_xg = test_x['poutcome']

In [166]:
train_x_nn = pd.read_csv('.//dataset//train_x_NN.csv')
train_y_nn = pd.read_csv('.//dataset//train_y_NN.csv')
test_x_nn = pd.read_csv('.//dataset//test_x_NN.csv')
train_x_nn_poutcome = train_x_nn.iloc[:,:48]
train_y_nn_poutcome = train_x_nn.iloc[:,48:]
test_x_nn_poutcome = test_x_nn.iloc[:,:48]
test_y_nn_poutcome = test_x_nn.iloc[:,48:]

In [75]:
train_y_nn_poutcome

,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0,1,0,0
1,0,0,0,1
2,1,0,0,0
3,1,0,0,0
4,0,0,0,1
...,...,...,...,...
27095,0,0,0,1
27096,0,0,0,1
27097,0,0,0,1
27098,0,0,0,1


In [9]:
def predict_cv_classfier(model,train_x, train_y, test_x):
    

    preds = []
    preds_test = []
    va_idxes = []


    kf = KFold(n_splits=4, shuffle=True, random_state= 71)
    for i , (tr_idx, va_idx) in enumerate (kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    tmp = np.stack(preds_test, axis =1)
    mode_test, mode_counts = mode(tmp, axis=1)

    preds_test = mode_test
    preds_size = preds_test.shape[0]
    preds_test = preds_test.reshape(preds_size,)
    
    return pred_train, preds_test

In [5]:
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []
    
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    for i , (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)
        
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]
    
    preds_test = np.mean(preds_test, axis=0)
    
    return pred_train, preds_test

In [8]:
from models import Model1NNsoftmax, Model1NN2softmax, Model1xgbsoftmax, Model1xgb2softmax,Model2KNN_p, Model1ramdom, Model3logistic

In [16]:
model_1NNa = Model1NNsoftmax()
pred_train_1NNa, preds_test_1NNa = predict_cv(model_1NNa, train_x_nn_poutcome,train_y_nn_poutcome,test_x_nn_poutcome )

model_1NNb = Model1NN2softmax()
pred_train_1NNb, preds_test_1NNb = predict_cv(model_1NNb, train_x_nn_poutcome, train_y_nn_poutcome,test_x_nn_poutcome)

model_1xgbc = Model1xgbsoftmax()
pred_train_1xgbc, preds_test_1xgbc = predict_cv(model_1xgbc, train_x_poutcome_xg,train_y_poutcome_xg,test_x_poutcome_xg)

model_1xgbd = Model1xgb2softmax()
pred_train_1xgbd, preds_teat_1xgbd = predict_cv(model_1xgbd, train_x_poutcome_xg, train_y_poutcome_xg,test_x_poutcome_xg)

model_1knne = Model2KNN_p()
pred_train_1knne, preds_test_1knne = predict_cv_classfier(model_1knne, train_x_poutcome_xg,train_y_poutcome_xg, test_x_poutcome_xg)


Epoch 1/100
159/159 [==============================] - 1s 4ms/step - loss: 0.4237 - accuracy: 0.8696 - val_loss: 0.3397 - val_accuracy: 0.8852
Epoch 2/100
159/159 [==============================] - 1s 5ms/step - loss: 0.3385 - accuracy: 0.8869 - val_loss: 0.3300 - val_accuracy: 0.8893
Epoch 3/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3193 - accuracy: 0.8909 - val_loss: 0.3251 - val_accuracy: 0.8928
Epoch 4/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3124 - accuracy: 0.8922 - val_loss: 0.3260 - val_accuracy: 0.8894
Epoch 5/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3070 - accuracy: 0.8942 - val_loss: 0.3230 - val_accuracy: 0.8939
Epoch 6/100
159/159 [==============================] - 1s 4ms/step - loss: 0.2975 - accuracy: 0.8948 - val_loss: 0.3240 - val_accuracy: 0.8903
Epoch 7/100
159/159 [==============================] - 1s 4ms/step - loss: 0.2907 - accuracy: 0.8974 - val_loss: 0.3233 - val_accuracy: 0.8912

159/159 [==============================] - 1s 4ms/step - loss: 0.2190 - accuracy: 0.9184 - val_loss: 0.3368 - val_accuracy: 0.8893
Epoch 1/100
159/159 [==============================] - 1s 5ms/step - loss: 0.4172 - accuracy: 0.8705 - val_loss: 0.3395 - val_accuracy: 0.8874
Epoch 2/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3364 - accuracy: 0.8878 - val_loss: 0.3335 - val_accuracy: 0.8872
Epoch 3/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3237 - accuracy: 0.8895 - val_loss: 0.3262 - val_accuracy: 0.8905
Epoch 4/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3134 - accuracy: 0.8931 - val_loss: 0.3214 - val_accuracy: 0.8914
Epoch 5/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3069 - accuracy: 0.8927 - val_loss: 0.3244 - val_accuracy: 0.8877
Epoch 6/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3014 - accuracy: 0.8950 - val_loss: 0.3240 - val_accuracy: 0.8899
Epoch 7/100

212/212 [==============================] - 1s 4ms/step - loss: 0.5021 - accuracy: 0.8502 - val_loss: 0.3562 - val_accuracy: 0.8825
Epoch 2/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3894 - accuracy: 0.8760 - val_loss: 0.3466 - val_accuracy: 0.8824
Epoch 3/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3762 - accuracy: 0.8801 - val_loss: 0.3375 - val_accuracy: 0.8850
Epoch 4/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3638 - accuracy: 0.8817 - val_loss: 0.3383 - val_accuracy: 0.8869
Epoch 5/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3550 - accuracy: 0.8832 - val_loss: 0.3325 - val_accuracy: 0.8884
Epoch 6/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3546 - accuracy: 0.8841 - val_loss: 0.3301 - val_accuracy: 0.8906
Epoch 7/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3459 - accuracy: 0.8841 - val_loss: 0.3268 - val_accuracy: 0.8914
Epoch 8/300

212/212 [==============================] - 1s 4ms/step - loss: 0.2783 - accuracy: 0.8996 - val_loss: 0.2937 - val_accuracy: 0.8976
Epoch 59/300
212/212 [==============================] - 1s 4ms/step - loss: 0.2761 - accuracy: 0.9008 - val_loss: 0.2930 - val_accuracy: 0.8974
Epoch 60/300
212/212 [==============================] - 1s 4ms/step - loss: 0.2735 - accuracy: 0.8992 - val_loss: 0.2924 - val_accuracy: 0.8976
Epoch 61/300
212/212 [==============================] - 1s 4ms/step - loss: 0.2746 - accuracy: 0.8999 - val_loss: 0.2892 - val_accuracy: 0.8979
Epoch 62/300
212/212 [==============================] - 1s 4ms/step - loss: 0.2764 - accuracy: 0.8990 - val_loss: 0.2901 - val_accuracy: 0.8990
Epoch 63/300
212/212 [==============================] - 1s 4ms/step - loss: 0.2734 - accuracy: 0.9006 - val_loss: 0.2911 - val_accuracy: 0.8964
Epoch 64/300
212/212 [==============================] - 1s 4ms/step - loss: 0.2722 - accuracy: 0.9027 - val_loss: 0.2902 - val_accuracy: 0.8987
Epoch

212/212 [==============================] - 1s 4ms/step - loss: 0.3198 - accuracy: 0.8901 - val_loss: 0.2992 - val_accuracy: 0.9008
Epoch 25/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3171 - accuracy: 0.8891 - val_loss: 0.3066 - val_accuracy: 0.8980
Epoch 26/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3159 - accuracy: 0.8896 - val_loss: 0.2987 - val_accuracy: 0.8985
Epoch 27/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3135 - accuracy: 0.8897 - val_loss: 0.3005 - val_accuracy: 0.8983
Epoch 28/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3164 - accuracy: 0.8914 - val_loss: 0.3004 - val_accuracy: 0.8982
Epoch 29/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3132 - accuracy: 0.8909 - val_loss: 0.2981 - val_accuracy: 0.8985
Epoch 30/300
212/212 [==============================] - 1s 4ms/step - loss: 0.3128 - accuracy: 0.8901 - val_loss: 0.2998 - val_accuracy: 0.8970
Epoch

...

[81]	train-merror:0.05073	eval-merror:0.08030
[82]	train-merror:0.05048	eval-merror:0.08030
[83]	train-merror:0.05009	eval-merror:0.08000
[84]	train-merror:0.04964	eval-merror:0.08000
[85]	train-merror:0.04930	eval-merror:0.08015
[86]	train-merror:0.04900	eval-merror:0.08044
[87]	train-merror:0.04895	eval-merror:0.08030
[88]	train-merror:0.04846	eval-merror:0.08015
[89]	train-merror:0.04812	eval-merror:0.08015
[90]	train-merror:0.04797	eval-merror:0.08000
[91]	train-merror:0.04758	eval-merror:0.07911
[92]	train-merror:0.04738	eval-merror:0.07941
[93]	train-merror:0.04679	eval-merror:0.07941
[94]	train-merror:0.04679	eval-merror:0.07956
[95]	train-merror:0.04640	eval-merror:0.07926
[96]	train-merror:0.04556	eval-merror:0.07956
[97]	train-merror:0.04536	eval-merror:0.07956
[98]	train-merror:0.04531	eval-merror:0.07941
[99]	train-merror:0.04487	eval-merror:0.07970
[100]	train-merror:0.04467	eval-merror:0.07941
[101]	train-merror:0.04453	eval-merror:0.07941
[102]	train-merror:0.04399	eval-

[112]	train-merror:0.06362	eval-merror:0.07852
Stopping. Best iteration:
[92]	train-merror:0.06760	eval-merror:0.07793

[11:13:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.09392	eval-merror:0.09077
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 20 rounds.
[1]	train-merror:0.09476	eval-merror:0.09107
[2]	train-merror:0.09368	eval-merror:0.09063
[3]	train-merror:0.09309	eval-merror:0.09063
[4]	train-merror:0.09333	eval-merror:0.08989
[5]	train-merror:0.09314	eval-merror:0.09048
[6]	train-merror:0.09289	eval-merror:0.09004
[7]	train-merror:0.09255	eval-merr

[6]	train-merror:0.08950	eval-merror:0.09491
[7]	train-merror:0.08984	eval-merror:0.09506
[8]	train-merror:0.08989	eval-merror:0.09491
[9]	train-merror:0.08969	eval-merror:0.09520
[10]	train-merror:0.08964	eval-merror:0.09506
[11]	train-merror:0.08979	eval-merror:0.09461
[12]	train-merror:0.08950	eval-merror:0.09476
[13]	train-merror:0.08954	eval-merror:0.09446
[14]	train-merror:0.08950	eval-merror:0.09476
[15]	train-merror:0.08994	eval-merror:0.09446
[16]	train-merror:0.08969	eval-merror:0.09520
[17]	train-merror:0.08979	eval-merror:0.09506
[18]	train-merror:0.08915	eval-merror:0.09594
[19]	train-merror:0.08950	eval-merror:0.09579
[20]	train-merror:0.08959	eval-merror:0.09550
[21]	train-merror:0.08930	eval-merror:0.09550
[22]	train-merror:0.08856	eval-merror:0.09565
[23]	train-merror:0.08866	eval-merror:0.09565
[24]	train-merror:0.08738	eval-merror:0.09446
[25]	train-merror:0.08699	eval-merror:0.09461
[26]	train-merror:0.08507	eval-merror:0.09284
[27]	train-merror:0.08487	eval-merror:

Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 20 rounds.
[1]	train-merror:0.09284	eval-merror:0.09476
[2]	train-merror:0.09284	eval-merror:0.09432
[3]	train-merror:0.09201	eval-merror:0.09520
[4]	train-merror:0.09181	eval-merror:0.09506
[5]	train-merror:0.09146	eval-merror:0.09594
[6]	train-merror:0.09171	eval-merror:0.09609
[7]	train-merror:0.09137	eval-merror:0.09520
[8]	train-merror:0.09087	eval-merror:0.09535
[9]	train-merror:0.09107	eval-merror:0.09476
[10]	train-merror:0.09097	eval-merror:0.09520
[11]	train-merror:0.09063	eval-merror:0.09506
[12]	train-merror:0.09053	eval-merror:0.09506
[13]	train-merror:0.09053	eval-merror:0.09506
[14]	train-merror:0.09028	eval-merror:0.09461
[15]	train-merror:0.09023	eval-merror:0.09432
[16]	train-merror:0.09019	eval-merror:0.09461
[17]	train-merror:0.09023	eval-merror:0.09402
[18]	train-merror:0.08999	eval-merror:0.09388
[19]	train-merror:0.09009	eval-mer

[175]	train-merror:0.05540	eval-merror:0.07956
[176]	train-merror:0.05545	eval-merror:0.07956
[177]	train-merror:0.05545	eval-merror:0.07941
[178]	train-merror:0.05515	eval-merror:0.07941
[179]	train-merror:0.05520	eval-merror:0.07970
[180]	train-merror:0.05501	eval-merror:0.07985
[181]	train-merror:0.05471	eval-merror:0.07985
[182]	train-merror:0.05422	eval-merror:0.07985
[183]	train-merror:0.05427	eval-merror:0.08000
Stopping. Best iteration:
[163]	train-merror:0.05737	eval-merror:0.07941



In [73]:
print(f'logloss: {log_loss(train_y_nn_poutcome, pred_train_1NNa, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_nn_poutcome, pred_train_1NNb, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_poutcome_xg, pred_train_1xgbc, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_poutcome_xg, pred_train_1xgbd, eps=1e-7):.4f}')
loss_c = pd.DataFrame(pred_train_1knne)
loss_c = pd.get_dummies(loss_c[0])
print(f'logloss: {log_loss(train_y_poutcome_xg, loss_c, eps=1e-7):.4f}')

logloss: 0.3178
logloss: 2.3796
logloss: 0.2190
logloss: 0.2182
logloss: 3.0327


In [79]:
model_1ramdomf = Model1ramdom()
pred_train_1ram1, preds_test_1ram1 = predict_cv(model_1ramdomf, train_x_poutcome_xg, train_y_poutcome_f, test_x_poutcome_xg)

pred_train_1ram2, preds_test_1ram2 = predict_cv(model_1ramdomf, train_x_poutcome_xg, train_y_poutcome_o, test_x_poutcome_xg)

pred_train_1ram3, preds_test_1ram3 = predict_cv(model_1ramdomf, train_x_poutcome_xg, train_y_poutcome_s, test_x_poutcome_xg)

pred_train_1ram4, preds_test_1ram4 = predict_cv(model_1ramdomf, train_x_poutcome_xg, train_y_poutcome_u, test_x_poutcome_xg)

In [80]:
print(f'logloss: {log_loss(train_y_poutcome_f, pred_train_1ram1, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_poutcome_o, pred_train_1ram2, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_poutcome_s, pred_train_1ram3, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_poutcome_u, pred_train_1ram4, eps=1e-7):.4f}')

logloss: 0.1789
logloss: 0.1160
logloss: 0.0722
logloss: 0.1690


In [140]:
a = np.split(pred_train_1NNa,4, axis=1)
df1 = pd.DataFrame(a[0])
df1.columns = ['1NNa_failure']
df1['1NNa_other'] = a[1]
df1['1NNa_success'] = a[2]
df1['1NNa_unknown'] = a[3]

a2 = np.split(preds_test_1NNa,4, axis=1)
df7 = pd.DataFrame(a2[0])
df7.columns = ['1NNa_failure_test']
df7['1NNa_other_test'] = a2[1]
df7['1NNa_success_test'] = a2[2]
df7['1NNa_unknown_test'] = a2[3]


In [141]:
b = np.split(pred_train_1NNb,4, axis=1)
df2 = pd.DataFrame(b[0])
df2.columns = ['1NNb_failure']
df2['1NNb_other'] = b[1]
df2['1NNb_success'] = b[2]
df2['1NNb_unknown'] = b[3]



b2 = np.split(preds_test_1NNb,4, axis=1)
df8 = pd.DataFrame(b2[0])
df8.columns = ['1NNb_failure_test']
df8['1NNb_other_test'] = b2[1]
df8['1NNb_success_test'] = b2[2]
df8['1NNb_unknown_test'] = b2[3]


In [143]:
c = np.split(pred_train_1xgbc,4, axis=1)
df3 = pd.DataFrame(c[0])
df3.columns = ['1xgbc_failure']
df3['1xgbc_other'] = c[1]
df3['1xgbc_success'] = c[2]
df3['1xgbc_unknown'] = c[3]



c2 = np.split(preds_test_1xgbc,4, axis=1)
df9 = pd.DataFrame(c2[0])
df9.columns = ['1xgbc_failure_test']
df9['1xgbc_other_test'] = c2[1]
df9['1xgbc_success_test'] = c2[2]
df9['1xgbc_unknown_test'] = c2[3]

In [144]:
d = np.split(pred_train_1xgbd,4, axis=1)
df4 = pd.DataFrame(d[0])
df4.columns = ['1xgbd_failure']
df4['1xgbd_other'] = d[1]
df4['1xgbd_success'] = d[2]
df4['1xgbd_unknown'] = d[3]



d2 = np.split(preds_teat_1xgbd,4, axis=1)
df10 = pd.DataFrame(d2[0])
df10.columns = ['1xgbd_failure_test']
df10['1xgbd_other_test'] = d2[1]
df10['1xgbd_success_test'] = d2[2]
df10['1xgbd_unknown_test'] = d2[3]

In [152]:
loss_c.columns = ['1knne_failure','1knne_other','1knne_success',
                 '1knne_unknown']
df5 = loss_c

df11 = pd.DataFrame(preds_test_1knne)
df11 = pd.get_dummies(df11[0])
df11.columns = ['1knne_failure_test','1knne_other_test','1knne_success_test',
                 '1knne_unknown_test']

In [153]:
df6 = pd.DataFrame(pred_train_1ram1)
df6.columns=['1ramdom_failure']
df6['1ramdom_other'] = pred_train_1ram2
df6['1ramdom_success'] = pred_train_1ram3
df6['1ramdom_unknown'] = pred_train_1ram4


df12 = pd.DataFrame(preds_test_1ram1)
df12.columns=['1ramdom_failure_test']
df12['1ramdom_other_test'] = preds_test_1ram2
df12['1ramdom_success_test'] = preds_test_1ram3
df12['1ramdom_unknown_test'] = preds_test_1ram4


In [156]:
train_x2 = pd.concat([df1,df2,df3,df4,df5,df6], axis = 1)
test_x2 = pd.concat([df7,df8,df9,df10,df11,df12], axis = 1)

In [ ]:
train_y_poutcome_f = train_y_poutcome_dummies[0]
train_y_poutcome_o = train_y_poutcome_dummies[1]
train_y_poutcome_s = train_y_poutcome_dummies[2]
train_y_poutcome_u = train_y_poutcome_dummies[3]

In [159]:
model2 = Model3logistic()
pred_train_2_f, preds_test_2_f = predict_cv(model2, train_x2, train_y_poutcome_f, test_x2)
pred_train_2_o, preds_test_2_o = predict_cv(model2, train_x2, train_y_poutcome_o, test_x2)
pred_train_2_s, preds_test_2_s = predict_cv(model2, train_x2, train_y_poutcome_s, test_x2)
pred_train_2_u, preds_test_2_u = predict_cv(model2, train_x2, train_y_poutcome_u, test_x2)

In [160]:
print(f'logloss: {log_loss(train_y_poutcome_f, pred_train_2_f, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_poutcome_o, pred_train_2_o, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_poutcome_s, pred_train_2_s, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y_poutcome_u, pred_train_2_u, eps=1e-7):.4f}')

logloss: 0.1392
logloss: 0.0886
logloss: 0.0489
logloss: 0.1320


In [162]:
df13 = pd.DataFrame(pred_train_2_f)
df13.columns=['stacking_f']
df13['stacking_o'] = pred_train_2_o
df13['stacking_s'] = pred_train_2_s
df13['stacking_u'] = pred_train_2_u



df14 = pd.DataFrame(preds_test_2_f)
df14.columns=['stacking_f_test']
df14['stacking_o_test'] = preds_test_2_o
df14['stacking_s_test'] = preds_test_2_s
df14['stacking_u_test'] = preds_test_2_u


In [173]:
train_x_new = train_x.drop(['poutcome'], axis = 1)
train_x_new = pd.concat([train_x_new, df13], axis=1)

train_x_nn_new = train_x_nn.iloc[:,:48]
train_x_nn_new = pd.concat([train_x_nn_new, df13], axis=1)

In [176]:
test_x_new = test_x.drop(['poutcome'], axis = 1)
test_x_new = pd.concat([test_x_new, df14], axis=1)

test_x_nn_new = test_x_nn.iloc[:,:48]
test_x_nn_new = pd.concat([test_x_nn_new, df14], axis=1)

In [181]:
train_x_new.to_csv('train_x_p_stack.csv')
test_x_new.to_csv('test_x_p_stack.csv')
train_x_nn.to_csv('train_x_nn_stack.csv')
test_x_nn_new.to_csv('test_x_nn_stack.csv')